In [1]:
import pandas as pd
import csv
import pprint as pprint
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
#import sklearn.datasets as dta
#from scipy.stats import linregress
#from scipy.stats import sem
import json
import requests
import os
from dotenv import load_dotenv
import gzip
# Load .env enviroment variables into the notebook
load_dotenv()
# Get the API key from the environment variable and store as Python variable
weather_api_key = os.getenv("weather_api_key")

In [13]:
# handling the large raw data file
filepath_redfin = "../data_sources/zip_code_market_tracker.tsv000"
redfinraw = pd.read_csv(filepath_redfin, delimiter = "\t")
redfinraw.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2014-08-01,2014-10-31,90,zip code,2,1920,f,Zip Code: 05489,NaN,Vermont,...,-0.325000,NaN,NaN,NaN,NaN,NaN,NaN,"Burlington, VT",15540,2024-03-10 14:36:40
1,2018-04-01,2018-06-30,90,zip code,2,18317,f,Zip Code: 43750,NaN,Ohio,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,"Cambridge, OH",15740,2024-03-10 14:36:40
2,2017-01-01,2017-03-31,90,zip code,2,13955,f,Zip Code: 33062,NaN,Florida,...,-0.600000,NaN,NaN,NaN,0.0,NaN,-0.500000,"Fort Lauderdale, FL",22744,2024-03-10 14:36:40
3,2015-12-01,2016-02-29,90,zip code,2,23759,f,Zip Code: 55437,NaN,Minnesota,...,-0.181818,NaN,NaN,NaN,0.0,-1.0,-0.166667,"Minneapolis, MN",33460,2024-03-10 14:36:40
4,2023-12-01,2024-02-29,90,zip code,2,25730,f,Zip Code: 59922,NaN,Montana,...,0.000000,NaN,NaN,NaN,1.0,1.0,1.000000,"Kalispell, MT",28060,2024-03-10 14:36:40


In [14]:
# listing the columns
cols = list(redfinraw.columns)
cols

['period_begin',
 'period_end',
 'period_duration',
 'region_type',
 'region_type_id',
 'table_id',
 'is_seasonally_adjusted',
 'region',
 'city',
 'state',
 'state_code',
 'property_type',
 'property_type_id',
 'median_sale_price',
 'median_sale_price_mom',
 'median_sale_price_yoy',
 'median_list_price',
 'median_list_price_mom',
 'median_list_price_yoy',
 'median_ppsf',
 'median_ppsf_mom',
 'median_ppsf_yoy',
 'median_list_ppsf',
 'median_list_ppsf_mom',
 'median_list_ppsf_yoy',
 'homes_sold',
 'homes_sold_mom',
 'homes_sold_yoy',
 'pending_sales',
 'pending_sales_mom',
 'pending_sales_yoy',
 'new_listings',
 'new_listings_mom',
 'new_listings_yoy',
 'inventory',
 'inventory_mom',
 'inventory_yoy',
 'months_of_supply',
 'months_of_supply_mom',
 'months_of_supply_yoy',
 'median_dom',
 'median_dom_mom',
 'median_dom_yoy',
 'avg_sale_to_list',
 'avg_sale_to_list_mom',
 'avg_sale_to_list_yoy',
 'sold_above_list',
 'sold_above_list_mom',
 'sold_above_list_yoy',
 'price_drops',
 'price_dro

In [24]:
#Define all zipcodes in seattle
allzipcodes = [
98102,
98103,
98107,
98109,
98112,
98115,
98117,
98119,
98125,
98133,
98177,
98105,
98106,
98108,
98112,
98116,
98118,
98122,
98126,
98134,
98136,
98144,
98146,
98178,
98199,
98101,
98104,
98113,
98121,
98141,
98154,
98161,
98165,
98170,
98174,
98175,
98181,
98185,
98190,
98191,
98194
]
# allzipcodes_str = [str(zipcode) for zipcode in allzipcodes]


In [25]:
#Change allzipcodes to match the headers in the dataframe by making them a string and appending

allzipcodes_format = []
for zipcode in allzipcodes:
    modified_zipcode = "Zip Code: " + str(zipcode)  
    allzipcodes_format.append(modified_zipcode)

print(allzipcodes_format)

['Zip Code: 98102', 'Zip Code: 98103', 'Zip Code: 98107', 'Zip Code: 98109', 'Zip Code: 98112', 'Zip Code: 98115', 'Zip Code: 98117', 'Zip Code: 98119', 'Zip Code: 98125', 'Zip Code: 98133', 'Zip Code: 98177', 'Zip Code: 98105', 'Zip Code: 98106', 'Zip Code: 98108', 'Zip Code: 98112', 'Zip Code: 98116', 'Zip Code: 98118', 'Zip Code: 98122', 'Zip Code: 98126', 'Zip Code: 98134', 'Zip Code: 98136', 'Zip Code: 98144', 'Zip Code: 98146', 'Zip Code: 98178', 'Zip Code: 98199', 'Zip Code: 98101', 'Zip Code: 98104', 'Zip Code: 98113', 'Zip Code: 98121', 'Zip Code: 98141', 'Zip Code: 98154', 'Zip Code: 98161', 'Zip Code: 98165', 'Zip Code: 98170', 'Zip Code: 98174', 'Zip Code: 98175', 'Zip Code: 98181', 'Zip Code: 98185', 'Zip Code: 98190', 'Zip Code: 98191', 'Zip Code: 98194']


In [27]:
#New Dataframe that only includes zipcodes that are in the Seattle area
metrodata = redfinraw[redfinraw["region"].isin(allzipcodes_format)]
metrodata

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
71,2018-03-01,2018-05-31,90,zip code,2,40833,f,Zip Code: 98121,NaN,Washington,...,-0.108458,NaN,NaN,NaN,0.777778,-0.011696,-0.116959,"Seattle, WA",42644,2024-03-10 14:36:40
1907,2014-05-01,2014-07-31,90,zip code,2,40816,f,Zip Code: 98103,NaN,Washington,...,0.004266,NaN,NaN,NaN,0.712121,-0.129984,0.109971,"Seattle, WA",42644,2024-03-10 14:36:40
3522,2014-09-01,2014-11-30,90,zip code,2,40865,f,Zip Code: 98175,NaN,Washington,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Seattle, WA",42644,2024-03-10 14:36:40
4375,2013-11-01,2014-01-31,90,zip code,2,40816,f,Zip Code: 98103,NaN,Washington,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,"Seattle, WA",42644,2024-03-10 14:36:40
4781,2018-09-01,2018-11-30,90,zip code,2,40834,f,Zip Code: 98122,NaN,Washington,...,-0.411765,NaN,NaN,NaN,0.333333,0.183333,-0.066667,"Seattle, WA",42644,2024-03-10 14:36:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7841828,2019-12-01,2020-02-29,90,zip code,2,40821,f,Zip Code: 98108,NaN,Washington,...,-0.083333,NaN,NaN,NaN,0.800000,0.400000,0.466667,"Seattle, WA",42644,2024-03-10 14:36:40
7842141,2016-03-01,2016-05-31,90,zip code,2,40850,f,Zip Code: 98146,NaN,Washington,...,0.233333,NaN,NaN,NaN,0.500000,-0.500000,-0.500000,"Seattle, WA",42644,2024-03-10 14:36:40
7842261,2022-03-01,2022-05-31,90,zip code,2,40825,f,Zip Code: 98112,NaN,Washington,...,0.293651,NaN,NaN,NaN,0.888889,0.111111,-0.111111,"Seattle, WA",42644,2024-03-10 14:36:40
7842290,2015-07-01,2015-09-30,90,zip code,2,40850,f,Zip Code: 98146,NaN,Washington,...,0.666667,NaN,NaN,NaN,0.500000,-0.166667,NaN,"Seattle, WA",42644,2024-03-10 14:36:40


In [ ]:
# The dataset has the following columns:

# period_begin
# period_end
# period_duration
# region_type
# region_type_id
# table_id
# is_seasonally_adjusted. (indicates if prices are seasonally adjusted; f represents False)
# region
# city
# state
# state_code
# property_type
# property_type_id
# median_sale_price
# median_sale_price_mom (median sale price changes month over month)
# median_sale_price_yoy (median sale price changes year over year)
# median_list_price
# median_list_price_mom (median list price changes month over month)
# median_list_price_yoy (median list price changes year over year)
# median_ppsf (median sale price per square foot)
# median_ppsf_mom (median sale price per square foot changes month over month)
# median_ppsf_yoy (median sale price per square foot changes year over year)
# median_list_ppsf (median list price per square foot)
# median_list_ppsf_mom (median list price per square foot changes month over month)
# median_list_ppsf_yoy. (median list price per square foot changes year over year)
# homes_sold (number of homes sold)
# homes_sold_mom (number of homes sold month over month)
# homes_sold_yoy (number of homes sold year over year)
# pending_sales
# pending_sales_mom
# pending_sales_yoy
# new_listings
# new_listings_mom
# new_listings_yoy
# inventory
# inventory_mom
# inventory_yoy
# months_of_supply
# months_of_supply_mom
# months_of_supply_yoy
# median_dom (median days on market until property is sold)
# median_dom_mom (median days on market changes month over month)
# median_dom_yoy (median days on market changes year over year)
# avg_sale_to_list (average sale price to list price ratio)
# avg_sale_to_list_mom (average sale price to list price ratio changes month over month)
# avg_sale_to_list_yoy (average sale price to list price ratio changes year over year)
# sold_above_list
# sold_above_list_mom
# sold_above_list_yoy
# price_drops
# price_drops_mom
# price_drops_yoy
# off_market_in_two_weeks (number of properties that will be taken off the market within 2 weeks)
# off_market_in_two_weeks_mom (changes in number of properties that will be taken off the market within 2 weeks, month over month)
# off_market_in_two_weeks_yoy (changes in number of properties that will be taken off the market within 2 weeks, year over year)
# parent_metro_region
# parent_metro_region_metro_code
# last_updated

In [43]:
#Begin narrowing down the colunms. all of these could be useful potentially but its a lot to track
metrodata2 = metrodata[[
    'period_begin',
    'period_end',
    'period_duration',
    'table_id',
    'region',
    'city',
    'state',
    'state_code',
    'property_type',
    'property_type_id',
    'median_sale_price',
    'median_sale_price_yoy',
    'median_list_price',
    'median_list_price_yoy',
    'median_ppsf',
    'median_ppsf_yoy',
    'median_list_ppsf',
    'median_list_ppsf_yoy',
    'homes_sold',
    'homes_sold_yoy',
    'pending_sales',
    'pending_sales_yoy',
    'new_listings',
    'new_listings_yoy',
    'inventory',
    'inventory_mom',
    'inventory_yoy',
    'months_of_supply',
    'months_of_supply_yoy',
    'avg_sale_to_list',
    'avg_sale_to_list_yoy',
    'sold_above_list',
    'sold_above_list_yoy',
    'price_drops',
    'price_drops_yoy',
    'parent_metro_region']]
# metrodata2.reset_index().sort_values(by="period_begin", ascending = True)
metrodata2.reset_index().head()
metrodata2 = metrodata2[[
    'period_begin',
    'period_end',
    'period_duration',
    'table_id',
    'region',
    'city',
    'state',
    'state_code',
    'property_type',
    'property_type_id',
    'median_sale_price',
    'median_sale_price_yoy',
    'median_list_price',
    'median_list_price_yoy',
    'median_ppsf',
    'median_ppsf_yoy',
    'median_list_ppsf',
    'median_list_ppsf_yoy',
    'homes_sold',
    'homes_sold_yoy',
    'pending_sales',
    'pending_sales_yoy',
    'new_listings',
    'new_listings_yoy',
    'inventory',
    'inventory_mom',
    'inventory_yoy',
    'months_of_supply',
    'months_of_supply_yoy',
    'avg_sale_to_list',
    'avg_sale_to_list_yoy',
    'sold_above_list',
    'sold_above_list_yoy',
    'price_drops',
    'price_drops_yoy',
    'parent_metro_region']]
metrodata2

,period_begin,period_end,period_duration,table_id,region,city,state,state_code,property_type,property_type_id,...,inventory_yoy,months_of_supply,months_of_supply_yoy,avg_sale_to_list,avg_sale_to_list_yoy,sold_above_list,sold_above_list_yoy,price_drops,price_drops_yoy,parent_metro_region
71,2018-03-01,2018-05-31,90,40833,Zip Code: 98121,NaN,Washington,WA,All Residential,-1,...,1.476190,NaN,NaN,1.037550,-0.010136,0.520408,-0.108458,NaN,NaN,"Seattle, WA"
1907,2014-05-01,2014-07-31,90,40816,Zip Code: 98103,NaN,Washington,WA,All Residential,-1,...,0.034884,NaN,NaN,1.030420,0.000531,0.545082,0.004266,NaN,NaN,"Seattle, WA"
3522,2014-09-01,2014-11-30,90,40865,Zip Code: 98175,NaN,Washington,WA,Townhouse,13,...,NaN,NaN,NaN,0.986301,NaN,0.000000,NaN,NaN,NaN,"Seattle, WA"
4375,2013-11-01,2014-01-31,90,40816,Zip Code: 98103,NaN,Washington,WA,Multi-Family (2-4 Unit),4,...,0.333333,NaN,NaN,0.969471,0.002038,0.000000,0.000000,NaN,NaN,"Seattle, WA"
4781,2018-09-01,2018-11-30,90,40834,Zip Code: 98122,NaN,Washington,WA,Townhouse,13,...,5.000000,NaN,NaN,0.989249,-0.036230,0.088235,-0.411765,NaN,NaN,"Seattle, WA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7841828,2019-12-01,2020-02-29,90,40821,Zip Code: 98108,NaN,Washington,WA,Townhouse,13,...,-0.333333,NaN,NaN,0.990874,0.008804,0.000000,-0.083333,NaN,NaN,"Seattle, WA"
7842141,2016-03-01,2016-05-31,90,40850,Zip Code: 98146,NaN,Washington,WA,Condo/Co-op,3,...,1.500000,NaN,NaN,1.007034,0.023603,0.400000,0.233333,NaN,NaN,"Seattle, WA"
7842261,2022-03-01,2022-05-31,90,40825,Zip Code: 98112,NaN,Washington,WA,Condo/Co-op,3,...,-0.636364,NaN,NaN,1.033408,0.025816,0.571429,0.293651,NaN,NaN,"Seattle, WA"
7842290,2015-07-01,2015-09-30,90,40850,Zip Code: 98146,NaN,Washington,WA,Condo/Co-op,3,...,-0.500000,NaN,NaN,1.026968,0.052054,0.666667,0.666667,NaN,NaN,"Seattle, WA"


In [39]:
# do we want to group by property type? (condo, townhouse, single-family?)
groupedbypropertytype = metrodata2.groupby("property_type")
groupedbypropertytype

In [9]:
# metrodata = metrodata[
#     'period_begin',
#     'period_end',
#     'period_duration',
#     'region_type',
#     'region_type_id',
#     'table_id',
#     'is_seasonally_adjusted',
#     'region',
#     'city',
#     'state',
#     'state_code',
#     'property_type',
#     'property_type_id',
#     'median_sale_price',
#     'median_sale_price_mom',
#     'median_sale_price_yoy',
#     'median_list_price',
#     'median_list_price_mom',
#     'median_list_price_yoy',
#     'median_ppsf',
#     'median_ppsf_mom',
#     'median_ppsf_yoy',
#     'median_list_ppsf',
#     'median_list_ppsf_mom',
#     'median_list_ppsf_yoy',
#     'homes_sold',
#     'homes_sold_mom',
#     'homes_sold_yoy',
#     'pending_sales',
#     'pending_sales_mom',
#     'pending_sales_yoy',
#     'new_listings',
#     'new_listings_mom',
#     'new_listings_yoy',
#     'inventory',
#     'inventory_mom',
#     'inventory_yoy',
#     'months_of_supply',
#     'months_of_supply_mom',
#     'months_of_supply_yoy',
#     'median_dom',
#     'median_dom_mom',
#     'median_dom_yoy',
#     'avg_sale_to_list',
#     'avg_sale_to_list_mom',
#     'avg_sale_to_list_yoy',
#     'sold_above_list',
#     'sold_above_list_mom',
#     'sold_above_list_yoy',
#     'price_drops',
#     'price_drops_mom',
#     'price_drops_yoy',
#     'off_market_in_two_weeks',
#     'off_market_in_two_weeks_mom',
#     'off_market_in_two_weeks_yoy',
#     'parent_metro_region',
#     'parent_metro_region_metro_code',
#     'last_updated']

In [73]:
# Save the data for the seattle metro area with only potentially useful series headers
metrodata2.to_csv("../data_sources/redfin_by_zipcode.csv", index=False)

In [74]:
filepath_metrodata2 = "../data_sources/redfin_by_zipcode.csv"
metrodata2 = pd.read_csv(filepath_metrodata2)
metrodata2.head()

,period_begin,period_end,region,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
0,2018-03-01,2018-05-31,Zip Code: 98121,All Residential,667500.0,625000.0,860.121526,98.0,145.0,52.0,1.037550,0.520408
1,2014-05-01,2014-07-31,Zip Code: 98103,All Residential,539000.0,535000.0,318.695652,244.0,283.0,89.0,1.030420,0.545082
2,2014-09-01,2014-11-30,Zip Code: 98175,Townhouse,360000.0,NaN,260.680666,1.0,NaN,NaN,0.986301,0.000000
3,2013-11-01,2014-01-31,Zip Code: 98103,Multi-Family (2-4 Unit),570500.0,850000.0,244.882917,2.0,1.0,4.0,0.969471,0.000000
4,2018-09-01,2018-11-30,Zip Code: 98122,Townhouse,742500.0,749000.0,546.105168,34.0,46.0,18.0,0.989249,0.088235


In [75]:
# metrodata2 = metrodata2[[
#     'period_begin',
#     'period_end',
#     'table_id',
#     'region',
#     'city',
#     'property_type',
#     'property_type_id',
#     'median_sale_price',
#     'median_sale_price_yoy',
#     'median_list_price',
#     'median_list_price_yoy',
#     'median_ppsf',
#     'median_ppsf_yoy',
#     'median_list_ppsf',
#     'median_list_ppsf_yoy',
#     'homes_sold',
#     'homes_sold_yoy',
#     'pending_sales',
#     'pending_sales_yoy',
#     'new_listings',
#     'new_listings_yoy',
#     'inventory',
#     'inventory_mom',
#     'inventory_yoy',
#     'avg_sale_to_list',
#     'avg_sale_to_list_yoy',
#     'sold_above_list',
#     'sold_above_list_yoy',
#     'parent_metro_region' 
# ]]


KeyError: "['table_id', 'city', 'property_type_id', 'median_sale_price_yoy', 'median_list_price_yoy', 'median_ppsf_yoy', 'median_list_ppsf', 'median_list_ppsf_yoy', 'homes_sold_yoy', 'pending_sales', 'pending_sales_yoy', 'new_listings_yoy', 'inventory_mom', 'inventory_yoy', 'avg_sale_to_list_yoy', 'sold_above_list_yoy', 'parent_metro_region'] not in index"

In [76]:
#Here are what might be the best data points in the dataframe
metrodata2 = metrodata2[[
    'period_begin',
    'period_end',
    'region',
    'property_type',
    'median_sale_price',
    'median_list_price',
    'median_ppsf',
    'homes_sold',
    'new_listings',
    'inventory',
    'avg_sale_to_list',
    'sold_above_list'
]]

In [84]:
#Change the period_end and period_start series to date time from string
metrodata2['period_begin'] = pd.to_datetime(metrodata2['period_begin'])
metrodata2['period_end'] = pd.to_datetime(metrodata2['period_end'])
# metrodata2.sort_values(by = 'period_end', ascending=True)
metrodata2

,period_begin,period_end,region,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
0,2018-03-01,2018-05-31,Zip Code: 98121,All Residential,667500.0,625000.0,860.121526,98.0,145.0,52.0,1.037550,0.520408
1,2014-05-01,2014-07-31,Zip Code: 98103,All Residential,539000.0,535000.0,318.695652,244.0,283.0,89.0,1.030420,0.545082
2,2014-09-01,2014-11-30,Zip Code: 98175,Townhouse,360000.0,NaN,260.680666,1.0,NaN,NaN,0.986301,0.000000
3,2013-11-01,2014-01-31,Zip Code: 98103,Multi-Family (2-4 Unit),570500.0,850000.0,244.882917,2.0,1.0,4.0,0.969471,0.000000
4,2018-09-01,2018-11-30,Zip Code: 98122,Townhouse,742500.0,749000.0,546.105168,34.0,46.0,18.0,0.989249,0.088235
...,...,...,...,...,...,...,...,...,...,...,...,...
17000,2019-12-01,2020-02-29,Zip Code: 98108,Townhouse,543750.0,541500.0,438.266981,8.0,16.0,8.0,0.990874,0.000000
17001,2016-03-01,2016-05-31,Zip Code: 98146,Condo/Co-op,178000.0,167500.0,154.782609,5.0,6.0,5.0,1.007034,0.400000
17002,2022-03-01,2022-05-31,Zip Code: 98112,Condo/Co-op,715500.0,659999.0,790.378007,21.0,25.0,4.0,1.033408,0.571429
17003,2015-07-01,2015-09-30,Zip Code: 98146,Condo/Co-op,249000.0,261500.0,271.077110,6.0,10.0,2.0,1.026968,0.666667


In [78]:
# Sort by period_end and group by unique zipcode
metrodata2_sorted =  metrodata2.sort_values(by='period_end')
groupedmetrodata2_sorted = metrodata2_sorted.groupby('region')
groupedmetrodata2_sorted

In [112]:
# Cool Function to chaneg the date time (dt) of period_end to the quarter
quarters = metrodata2['period_end'].dt.to_period('Q').unique()

# Empty dictionary to store mean_median_sale_price for each quarter and each group(zipcode)

zipcode_and_mean_dict = {zipcode: [] for zipcode in groupedmetrodata2_sorted.groups.keys()}

#For-loop to go through the grouped data to find the mean_median_sale_price for every quarter
for zipcode, group in groupedmetrodata2_sorted:
    for quarter in quarters:
        mean_median_sale_price = group[group['period_end'].dt.to_period('Q') == quarter]['median_sale_price'].mean()
        zipcode_and_mean_dict[zipcode].append(mean_median_sale_price)

#Put this data from the loop into a Dataframe with the dictionary keys and datetime
mean_sales_df = pd.DataFrame(zipcode_and_mean_dict, index=quarters)

# Transpose Dataframe. The df.T will switch columns and rows
mean_sales_df = mean_sales_df.T

In [115]:
#Drop the decimal places and make it an int instead of float
mean_sales_df = mean_sales_df.dropna().round(0).astype(int)
mean_sales_df = mean_sales_df.sort_index(axis=1)
mean_sales_df

,2012Q1,2012Q2,2012Q3,2012Q4,2013Q1,2013Q2,2013Q3,2013Q4,2014Q1,2014Q2,...,2021Q4,2022Q1,2022Q2,2022Q3,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4,2024Q1
Zip Code: 98101,632000,480167,503150,499833,578000,531667,421683,473167,697667,528167,...,863425,942889,745000,631167,646454,538833,658333,652128,600278,850000
Zip Code: 98102,478400,469706,472678,510033,481990,563115,574748,713090,588084,610917,...,1033128,1118357,1077755,1052790,991762,971769,1000496,1043535,1071667,829414
Zip Code: 98103,396300,440550,407155,394756,408807,449780,443280,459820,472632,460561,...,788623,847252,875080,868017,905467,851765,938605,942392,912098,880175
Zip Code: 98104,324833,302212,206200,246000,301667,354000,311833,491003,770344,533264,...,562483,393975,420808,419158,376642,415892,413967,408892,381217,453400
Zip Code: 98105,458600,508259,500059,483693,441060,504517,541387,491943,533933,538900,...,899382,1026050,1227836,1190863,1051495,975626,1057597,985137,931402,1015833
Zip Code: 98106,205500,179825,201010,198691,228532,232750,258152,268311,237871,264536,...,648380,640167,649590,694993,611561,615547,639659,617168,611327,660428
Zip Code: 98107,374950,385832,416759,426280,411113,422800,460415,465433,484125,496783,...,836380,933493,981430,888598,763613,851433,880923,913830,1061333,1088500
Zip Code: 98108,206250,238739,264486,265780,280402,288827,312840,307200,320343,312250,...,605393,717614,696830,631830,700550,602120,618758,681423,709951,727073
Zip Code: 98109,457250,492462,474910,509993,540981,512631,511593,499143,527750,592986,...,1080148,941621,1124170,1074154,942083,941055,1003354,1210531,1111392,1043325
Zip Code: 98112,599811,578349,545300,682722,691416,670786,671330,628355,623583,676000,...,1130128,1254533,1338163,1288017,1183615,1131194,1100720,1265363,1202545,1111584


In [117]:
# Save the data for the Average_Median_Sale_Price of homes in seattle zipcodes by quarter
mean_sales_df.to_csv("../data_sources/average_median_sale_price_by_quarter.csv", index=True)

In [118]:
filepath_metrodata2 = "../data_sources/redfin_by_zipcode.csv"
metrodata2 = pd.read_csv(filepath_metrodata2)
metrodata2.head()

,period_begin,period_end,region,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
0,2018-03-01,2018-05-31,Zip Code: 98121,All Residential,667500.0,625000.0,860.121526,98.0,145.0,52.0,1.037550,0.520408
1,2014-05-01,2014-07-31,Zip Code: 98103,All Residential,539000.0,535000.0,318.695652,244.0,283.0,89.0,1.030420,0.545082
2,2014-09-01,2014-11-30,Zip Code: 98175,Townhouse,360000.0,NaN,260.680666,1.0,NaN,NaN,0.986301,0.000000
3,2013-11-01,2014-01-31,Zip Code: 98103,Multi-Family (2-4 Unit),570500.0,850000.0,244.882917,2.0,1.0,4.0,0.969471,0.000000
4,2018-09-01,2018-11-30,Zip Code: 98122,Townhouse,742500.0,749000.0,546.105168,34.0,46.0,18.0,0.989249,0.088235


In [119]:
metrodata2 = metrodata2[[
    'period_begin',
    'period_end',
    'region',
    'property_type',
    'median_sale_price',
    'median_list_price',
    'median_ppsf',
    'homes_sold',
    'new_listings',
    'inventory',
    'avg_sale_to_list',
    'sold_above_list'
]]

In [135]:
property_types = metrodata2['property_type'].unique()
print(property_types)

['All Residential' 'Townhouse' 'Multi-Family (2-4 Unit)' 'Condo/Co-op'
 'Single Family Residential']


In [120]:
#Change the period_end and period_start series to date time from string
metrodata2['period_begin'] = pd.to_datetime(metrodata2['period_begin'])
metrodata2['period_end'] = pd.to_datetime(metrodata2['period_end'])
# metrodata2.sort_values(by = 'period_end', ascending=True)
metrodata2

,period_begin,period_end,region,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
0,2018-03-01,2018-05-31,Zip Code: 98121,All Residential,667500.0,625000.0,860.121526,98.0,145.0,52.0,1.037550,0.520408
1,2014-05-01,2014-07-31,Zip Code: 98103,All Residential,539000.0,535000.0,318.695652,244.0,283.0,89.0,1.030420,0.545082
2,2014-09-01,2014-11-30,Zip Code: 98175,Townhouse,360000.0,NaN,260.680666,1.0,NaN,NaN,0.986301,0.000000
3,2013-11-01,2014-01-31,Zip Code: 98103,Multi-Family (2-4 Unit),570500.0,850000.0,244.882917,2.0,1.0,4.0,0.969471,0.000000
4,2018-09-01,2018-11-30,Zip Code: 98122,Townhouse,742500.0,749000.0,546.105168,34.0,46.0,18.0,0.989249,0.088235
...,...,...,...,...,...,...,...,...,...,...,...,...
17000,2019-12-01,2020-02-29,Zip Code: 98108,Townhouse,543750.0,541500.0,438.266981,8.0,16.0,8.0,0.990874,0.000000
17001,2016-03-01,2016-05-31,Zip Code: 98146,Condo/Co-op,178000.0,167500.0,154.782609,5.0,6.0,5.0,1.007034,0.400000
17002,2022-03-01,2022-05-31,Zip Code: 98112,Condo/Co-op,715500.0,659999.0,790.378007,21.0,25.0,4.0,1.033408,0.571429
17003,2015-07-01,2015-09-30,Zip Code: 98146,Condo/Co-op,249000.0,261500.0,271.077110,6.0,10.0,2.0,1.026968,0.666667


In [136]:
# Sort by period_end and group by unique zipcode
metrodata2['inventory'] = metrodata2['inventory'].fillna(0)
metrodata2_sorted =  metrodata2.sort_values(by='period_end')
groupedmetrodata2_sorted = metrodata2_sorted.groupby('region')
groupedmetrodata2_sorted.head()

,period_begin,period_end,region,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
16556,2012-01-01,2012-03-31,Zip Code: 98105,All Residential,556000.0,539950.0,270.175439,47.0,75.0,41.0,0.975642,0.148936
12764,2012-01-01,2012-03-31,Zip Code: 98105,Single Family Residential,662500.0,599500.0,271.429382,36.0,56.0,21.0,0.976863,0.194444
5438,2012-01-01,2012-03-31,Zip Code: 98116,Condo/Co-op,252250.0,235000.0,250.265055,18.0,27.0,23.0,0.962819,0.055556
5427,2012-01-01,2012-03-31,Zip Code: 98144,Condo/Co-op,185750.0,244900.0,221.331884,6.0,24.0,33.0,0.994658,0.166667
16913,2012-01-01,2012-03-31,Zip Code: 98199,Single Family Residential,530000.0,594500.0,250.000000,45.0,68.0,58.0,0.983099,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...
11881,2017-01-01,2017-03-31,Zip Code: 98185,Single Family Residential,260000.0,NaN,317.073171,1.0,NaN,0.0,0.945455,0.000000
8325,2017-01-01,2017-03-31,Zip Code: 98185,All Residential,260000.0,NaN,317.073171,1.0,NaN,0.0,0.945455,0.000000
8112,2017-02-01,2017-04-30,Zip Code: 98185,All Residential,260000.0,NaN,317.073171,1.0,NaN,0.0,0.945455,0.000000
3219,2017-02-01,2017-04-30,Zip Code: 98185,Single Family Residential,260000.0,NaN,317.073171,1.0,NaN,0.0,0.945455,0.000000


In [144]:
# Cool Function to chaneg the date time (dt) of period_end to the month
months = metrodata2['period_end'].dt.to_period('M').unique()

# Empty dictionary to store inventory_available_houses for each month and each group(zipcode)

zipcode_and_avail_houses_dict = {zipcode: [] for zipcode in groupedmetrodata2_sorted.groups.keys()}

#For-loop to go through the grouped data to find the zipcode_and_#avail_houses for every month
for zipcode, group in groupedmetrodata2_sorted:
    for month in months:
        available_houses = group[group['period_end'].dt.to_period('M') == month]['inventory'].sum()
        zipcode_and_avail_houses_dict[zipcode].append(available_houses)

#Put this data from the loop into a Dataframe with the available_houses for each zipcode by datetime
total_house_inventory = pd.DataFrame(zipcode_and_avail_houses_dict, index=months)

# Transpose Dataframe. The df.T will switch columns and rows
total_house_inventory = total_house_inventory.T
total_house_inventory = total_house_inventory.sort_index(axis=1)

In [146]:


total_house_inventory.head()

,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,...,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02
Zip Code: 98101,107.0,119.0,105.0,119.0,116.0,106.0,120.0,96.0,86.0,90.0,...,118.0,140.0,126.0,134.0,150.0,142.0,118.0,110.0,106.0,118.0
Zip Code: 98102,118.0,138.0,124.0,148.0,132.0,118.0,122.0,94.0,102.0,88.0,...,108.0,109.0,103.0,126.0,145.0,124.0,94.0,64.0,73.0,88.0
Zip Code: 98103,178.0,188.0,192.0,190.0,208.0,200.0,226.0,162.0,130.0,94.0,...,108.0,136.0,122.0,146.0,184.0,174.0,122.0,72.0,80.0,94.0
Zip Code: 98104,36.0,34.0,40.0,47.0,35.0,33.0,33.0,33.0,27.0,25.0,...,64.0,66.0,76.0,68.0,76.0,72.0,70.0,52.0,56.0,54.0
Zip Code: 98105,82.0,86.0,102.0,110.0,130.0,110.0,98.0,94.0,84.0,66.0,...,76.0,78.0,92.0,88.0,108.0,84.0,70.0,53.0,75.0,70.0


In [147]:
# Save the data for the total_housing_availability of homes in seattle zipcodes by month
total_house_inventory.to_csv("../data_frames/total_housing_availability_by_month.csv", index=True)

In [170]:
filepath_metrodata2 = "../data_sources/redfin_by_zipcode.csv"
metrodata2 = pd.read_csv(filepath_metrodata2)
metrodata2.head()

,period_begin,period_end,region,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
0,2018-03-01,2018-05-31,Zip Code: 98121,All Residential,667500.0,625000.0,860.121526,98.0,145.0,52.0,1.037550,0.520408
1,2014-05-01,2014-07-31,Zip Code: 98103,All Residential,539000.0,535000.0,318.695652,244.0,283.0,89.0,1.030420,0.545082
2,2014-09-01,2014-11-30,Zip Code: 98175,Townhouse,360000.0,NaN,260.680666,1.0,NaN,NaN,0.986301,0.000000
3,2013-11-01,2014-01-31,Zip Code: 98103,Multi-Family (2-4 Unit),570500.0,850000.0,244.882917,2.0,1.0,4.0,0.969471,0.000000
4,2018-09-01,2018-11-30,Zip Code: 98122,Townhouse,742500.0,749000.0,546.105168,34.0,46.0,18.0,0.989249,0.088235


In [171]:
metrodata2 = metrodata2[[
    'period_begin',
    'period_end',
    'region',
    'property_type',
    'median_sale_price',
    'median_list_price',
    'median_ppsf',
    'homes_sold',
    'new_listings',
    'inventory',
    'avg_sale_to_list',
    'sold_above_list'
]]

In [172]:
property_types = metrodata2['property_type'].unique()
print(property_types)

['All Residential' 'Townhouse' 'Multi-Family (2-4 Unit)' 'Condo/Co-op'
 'Single Family Residential']


In [173]:
#Change the period_end and period_start series to date time from string
metrodata2['period_begin'] = pd.to_datetime(metrodata2['period_begin'])
metrodata2['period_end'] = pd.to_datetime(metrodata2['period_end'])
# metrodata2.sort_values(by = 'period_end', ascending=True)
metrodata2

,period_begin,period_end,region,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
0,2018-03-01,2018-05-31,Zip Code: 98121,All Residential,667500.0,625000.0,860.121526,98.0,145.0,52.0,1.037550,0.520408
1,2014-05-01,2014-07-31,Zip Code: 98103,All Residential,539000.0,535000.0,318.695652,244.0,283.0,89.0,1.030420,0.545082
2,2014-09-01,2014-11-30,Zip Code: 98175,Townhouse,360000.0,NaN,260.680666,1.0,NaN,NaN,0.986301,0.000000
3,2013-11-01,2014-01-31,Zip Code: 98103,Multi-Family (2-4 Unit),570500.0,850000.0,244.882917,2.0,1.0,4.0,0.969471,0.000000
4,2018-09-01,2018-11-30,Zip Code: 98122,Townhouse,742500.0,749000.0,546.105168,34.0,46.0,18.0,0.989249,0.088235
...,...,...,...,...,...,...,...,...,...,...,...,...
17000,2019-12-01,2020-02-29,Zip Code: 98108,Townhouse,543750.0,541500.0,438.266981,8.0,16.0,8.0,0.990874,0.000000
17001,2016-03-01,2016-05-31,Zip Code: 98146,Condo/Co-op,178000.0,167500.0,154.782609,5.0,6.0,5.0,1.007034,0.400000
17002,2022-03-01,2022-05-31,Zip Code: 98112,Condo/Co-op,715500.0,659999.0,790.378007,21.0,25.0,4.0,1.033408,0.571429
17003,2015-07-01,2015-09-30,Zip Code: 98146,Condo/Co-op,249000.0,261500.0,271.077110,6.0,10.0,2.0,1.026968,0.666667


In [191]:
# Sort by period_end and group by unique zipcode AND type of property
metrodata2['inventory'] = metrodata2['inventory'].fillna(0)
metrodata2_sorted =  metrodata2.sort_values(by='period_end')
groupedmetrodata2_sorted = metrodata2_sorted.groupby(['region', 'property_type'])
groupedmetrodata2_sorted.head()
# df.groupby(col1)[col2] 

,period_begin,period_end,region,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
16556,2012-01-01,2012-03-31,Zip Code: 98105,All Residential,556000.0,539950.0,270.175439,47.0,75.0,41.0,0.975642,0.148936
12764,2012-01-01,2012-03-31,Zip Code: 98105,Single Family Residential,662500.0,599500.0,271.429382,36.0,56.0,21.0,0.976863,0.194444
5438,2012-01-01,2012-03-31,Zip Code: 98116,Condo/Co-op,252250.0,235000.0,250.265055,18.0,27.0,23.0,0.962819,0.055556
5427,2012-01-01,2012-03-31,Zip Code: 98144,Condo/Co-op,185750.0,244900.0,221.331884,6.0,24.0,33.0,0.994658,0.166667
16913,2012-01-01,2012-03-31,Zip Code: 98199,Single Family Residential,530000.0,594500.0,250.000000,45.0,68.0,58.0,0.983099,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...
3932,2021-03-01,2021-05-31,Zip Code: 98185,Single Family Residential,459000.0,459000.0,628.767123,1.0,1.0,0.0,1.000000,0.000000
3274,2021-04-01,2021-06-30,Zip Code: 98185,Single Family Residential,459000.0,459000.0,628.767123,1.0,1.0,0.0,1.000000,0.000000
1292,2021-04-01,2021-06-30,Zip Code: 98185,All Residential,459000.0,459000.0,628.767123,1.0,1.0,0.0,1.000000,0.000000
5674,2021-09-01,2021-11-30,Zip Code: 98101,Townhouse,1387500.0,1345000.0,685.798157,2.0,4.0,1.0,1.030466,0.500000


In [192]:
groupedmetrodata2_sorted.fillna(0, inplace=True)

""


In [200]:
# Cool Function to chaneg the date time (dt) of period_end to the month
months = metrodata2['period_end'].dt.to_period('M').unique()

# Empty dictionary to store available houses for each property type for each month and each group(zipcode)
avail_houses_by_type_dict = {zipcode: [] for zipcode in groupedmetrodata2_sorted.groups.keys()}



#For-loop to go through the grouped data to find the mean_median_sale_price for every quarter
for (zipcode, property_type), group in groupedmetrodata2_sorted:
    avail_houses_by_type_dict[(zipcode, property_type)] = []
    for month in months:
        available_houses = group[group['period_end'].dt.to_period('M') == month]['inventory'].sum()
        avail_houses_by_type_dict[(zipcode, property_type)].append(available_houses)

        
index = pd.MultiIndex.from_tuples(avail_houses_by_type_dict.keys(), names=['zipcode', 'property_type'])


#Put this data from the loop into a Dataframe with the number of available_house for each propertytype and zipcode by month
total_house_inventory_for_property_type = pd.DataFrame(avail_houses_by_type_dict.values(), index=index, columns=months)


# Transpose Dataframe. The df.T will switch columns and rows
# total_house_inventory_for_property_type = total_house_inventory_for_property_type.T

#Sort by the multiindex
total_house_inventory_for_property_type = total_house_inventory_for_property_type.sort_index()



In [202]:
total_house_inventory_for_property_type

2018-05  2014-07  2014-11  2014-01  \
zipcode         property_type                                                   
Zip Code: 98101 All Residential               61.0     39.0     19.0     19.0   
                Condo/Co-op                   61.0     39.0     19.0     19.0   
                Single Family Residential      0.0      0.0      0.0      0.0   
                Townhouse                      0.0      0.0      0.0      0.0   
Zip Code: 98102 All Residential               36.0     50.0     33.0     37.0   
...                                            ...      ...      ...      ...   
Zip Code: 98199 All Residential               40.0     84.0     39.0     44.0   
                Condo/Co-op                    7.0     12.0      3.0      1.0   
                Multi-Family (2-4 Unit)        2.0      5.0      0.0      0.0   
                Single Family Residential     29.0     59.0     34.0     38.0   
                Townhouse                      2.0      8.0      2.0      3.0   

                                           2018-11  2022-03  2020-09  2012-09  \
zipcode         property_type                                                   
Zip Code: 98101 All Residential               50.0     49.0    130.0     60.0   
                Condo/Co-op                   50.0     49.0    130.0     60.0   
                Single Family Residential      0.0      0.0      0.0      0.0   
                Townhouse                      0.0      0.0      0.0      0.0   
Zip Code: 98102 All Residential               58.0     31.0    120.0     61.0   
...                                            ...      ...      ...      ...   
Zip Code: 98199 All Residential               55.0     18.0     65.0     87.0   
                Condo/Co-op                   13.0      3.0     11.0     15.0   
                Multi-Family (2-4 Unit)        1.0      0.0      3.0      2.0   
                Single Family Residential     32.0     12.0     42.0     59.0   
                Townhouse                      9.0      3.0      9.0     11.0   

                                           2014-06  2023-12  ...  2018-01  \
zipcode         property_type                                ...            
Zip Code: 98101 All Residential               31.0     55.0  ...     18.0   
                Condo/Co-op                   31.0     55.0  ...     18.0   
                Single Family Residential      0.0      0.0  ...      0.0   
                Townhouse                      0.0      0.0  ...      0.0   
Zip Code: 98102 All Residential               51.0     32.0  ...     20.0   
...                                            ...      ...  ...      ...   
Zip Code: 98199 All Residential               70.0     21.0  ...     21.0   
                Condo/Co-op                   11.0      4.0  ...      5.0   
                Multi-Family (2-4 Unit)        5.0      0.0  ...      0.0   
                Single Family Residential     49.0     12.0  ...     15.0   
                Townhouse                      5.0      5.0  ...      1.0   

                                           2015-08  2022-02  2020-11  2013-10  \
zipcode         property_type                                                   
Zip Code: 98101 All Residential               57.0     44.0     89.0     38.0   
                Condo/Co-op                   57.0     44.0     89.0     38.0   
                Single Family Residential      0.0      0.0      0.0      0.0   
                Townhouse                      0.0      0.0      0.0      0.0   
Zip Code: 98102 All Residential               35.0     29.0    100.0     57.0   
...                                            ...      ...      ...      ...   
Zip Code: 98199 All Residential               37.0     13.0     61.0     76.0   
                Condo/Co-op                    4.0      4.0     13.0     13.0   
                Multi-Family (2-4 Unit)        1.0      0.0      4.0      2.0   
                Single Family Residential 

In [203]:
# Save the data for the housing_availability_by_zipcode_property_type of homes in seattle zipcodes by month
total_house_inventory_for_property_type.to_csv("../data_frames/housing_availability_by_zip_and_property_type.csv", index=True)